<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/hybrid_qnn_final_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane torch tqdm

In [ ]:
#!/usr/bin/env python3
"""
hybrid_qnn_final.py

A fully working hybrid quantum–classical parity learner in PennyLane + PyTorch.
This script avoids Pennylane‐transform issues by using a simple Python loop
over the batch (no @batch_params decorator). It includes:

– AngleEmbedding + StronglyEntanglingLayers ansatz
– Per‐sample QNode calls + dtype alignment
– Classical hidden layer + dropout + weight decay
– Gradient clipping + AdamW + cosine‐annealing LR
"""

import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

# -----------------------------------------------------------------------------
# 1. Hybrid Quantum-Classical Model
# -----------------------------------------------------------------------------
class HybridQuantumNet(nn.Module):
    def __init__(
        self,
        n_qubits: int   = 4,
        n_layers: int   = 2,
        hidden_dim: int = 8,
        lr: float       = 0.01,
        weight_decay: float = 1e-3,
    ):
        super().__init__()
        self.n_qubits = n_qubits

        # (a) PennyLane device
        self.dev = qml.device("default.qubit", wires=self.n_qubits)

        # (b) Variational circuit definition
        def circuit(inputs, weights):
            # inputs: 1D tensor of length n_qubits
            qml.templates.AngleEmbedding(
                inputs, wires=range(self.n_qubits), rotation="Y"
            )
            qml.templates.StronglyEntanglingLayers(
                weights, wires=range(self.n_qubits)
            )
            return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]

        # (c) Wrap as a Torch‐compatible QNode
        self.qnode = qml.QNode(
            circuit,
            self.dev,
            interface="torch",
            diff_method="backprop",
        )

        # (d) Trainable quantum weights
        weight_shapes = (n_layers, n_qubits, 3)
        self.q_weights = nn.Parameter(
            torch.randn(*weight_shapes) * 0.1
        )

        # (e) Classical post‐processing head
        self.classical = nn.Sequential(
            nn.Linear(self.n_qubits, hidden_dim),
            nn.Dropout(p=0.2),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1),
        )

        # (f) Optimizer + scheduler
        self.optimizer = optim.AdamW(
            self.parameters(),
            lr=lr,
            weight_decay=weight_decay,
        )
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=50, eta_min=1e-4
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (batch_size, n_qubits) float32 angles
        returns: (batch_size,) float32 probabilities
        """
        q_out = []
        for sample in x:
            # 1-run QNode per sample → list of n_qubits scalars
            expvals = self.qnode(sample, self.q_weights)

            # ensure a real tensor and align dtype
            if not isinstance(expvals, torch.Tensor):
                expvals = torch.stack(expvals)
            expvals = expvals.to(x.dtype)

            q_out.append(expvals)

        # stack to (batch_size, n_qubits)
        q_out = torch.stack(q_out, dim=0)

        # classical head + sigmoid → (batch_size,)
        logits = self.classical(q_out).squeeze(-1)
        return torch.sigmoid(logits)

    def step_scheduler(self):
        self.scheduler.step()


# -----------------------------------------------------------------------------
# 2. Synthetic Parity Dataset
# -----------------------------------------------------------------------------
def make_parity_dataset(
    n_samples: int = 1200,
    n_qubits:   int = 4,
    seed:       int = 42,
):
    torch.manual_seed(seed)
    bits   = torch.randint(0, 2, (n_samples, n_qubits)).float()
    angles = bits * torch.pi
    labels = (bits.sum(dim=1) % 2).float()
    return angles, labels


# -----------------------------------------------------------------------------
# 3. Training & Evaluation
# -----------------------------------------------------------------------------
def train_epoch(model, loader, criterion, max_grad_norm=1.0):
    model.train()
    total_loss = 0.0

    for Xb, yb in tqdm(loader, desc="Training", leave=False):
        model.optimizer.zero_grad()
        preds = model(Xb)
        loss  = criterion(preds, yb)
        loss.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        model.optimizer.step()
        total_loss += loss.item() * Xb.size(0)

    model.step_scheduler()
    return total_loss / len(loader.dataset)


def eval_epoch(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for Xb, yb in loader:
            preds     = model(Xb)
            predicted = (preds >= 0.5).float()
            correct  += (predicted == yb).sum().item()
            total    += Xb.size(0)
    return correct / total


# -----------------------------------------------------------------------------
# 4. Main
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    # Hyperparameters
    N_QUBITS    = 4
    N_LAYERS    = 2
    HIDDEN_DIM  = 8
    N_SAMPLES   = 1200
    BATCH_SIZE  = 64
    LR          = 0.01
    W_DECAY     = 1e-3
    EPOCHS      = 30
    TRAIN_SPLIT = 1000

    # Prepare data
    angles, labels = make_parity_dataset(N_SAMPLES, N_QUBITS)
    ds             = TensorDataset(angles, labels)
    train_ds, val_ds = torch.utils.data.random_split(
        ds, [TRAIN_SPLIT, N_SAMPLES - TRAIN_SPLIT]
    )
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)

    # Initialize model & loss
    model     = HybridQuantumNet(
        n_qubits=N_QUBITS,
        n_layers=N_LAYERS,
        hidden_dim=HIDDEN_DIM,
        lr=LR,
        weight_decay=W_DECAY,
    )
    criterion = nn.BCELoss()

    # Training loop
    for epoch in range(1, EPOCHS + 1):
        train_loss = train_epoch(model, train_loader, criterion)
        val_acc    = eval_epoch(model, val_loader)
        print(
            f"Epoch {epoch:2d} | "
            f"Train Loss: {train_loss:.4f} | "
            f"Val Acc: {val_acc:.3f}"
        )

    # Final evaluation
    final_acc = eval_epoch(model, val_loader)
    print(f"\nFinal Validation Accuracy: {final_acc:.3f}")